In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

data_path = os.path.join("..", "data")
raw_data_path = os.path.join(data_path, "bronze")
raw_data_file_path = os.path.join(raw_data_path, "ncr_ride_bookings.csv")
processed_data_path = os.path.join(data_path, "silver")
os.makedirs(raw_data_path, exist_ok=True)
os.makedirs(processed_data_path, exist_ok=True)

# Preprocessing

In [ ]:
df_raw = pd.read_csv(raw_data_file_path)


def rename_columns(df):
    """Apply function to lowercase all columns and replace spaces with underscores"""
    new_df = df.copy()

    new_df.columns = new_df.columns.str.lower().str.replace(" ", "_")
    return new_df


df = rename_columns(df_raw)

In [ ]:
def cast_to_dtypes(df):
    new_df = df.copy()
    new_df["date"] = pd.to_datetime(new_df["date"], errors="coerce")
    new_df["time"] = pd.to_datetime(new_df["time"], format="%H:%M:%S", errors="coerce").dt.time
    # combine date and time columns to datetime
    new_df["datetime"] = pd.to_datetime(new_df["date"].astype(str) + " " + new_df["time"].astype(str), errors="coerce")

    new_df["booking_id"] = new_df["booking_id"].astype(pd.StringDtype())
    new_df["booking_status"] = new_df["booking_status"].astype("category")
    new_df["customer_id"] = new_df["customer_id"].astype(pd.StringDtype())
    new_df["vehicle_type"] = new_df["vehicle_type"].astype("category")
    new_df["pickup_location"] = new_df["pickup_location"].astype(pd.StringDtype())
    new_df["drop_location"] = new_df["drop_location"].astype("category")
    new_df["avg_vtat"] = pd.to_numeric(new_df["avg_vtat"], errors="coerce")
    new_df["avg_ctat"] = pd.to_numeric(new_df["avg_ctat"], errors="coerce")

    # Add these columns and their types
    new_df["reason_for_cancelling_by_customer"] = new_df["reason_for_cancelling_by_customer"].astype("category")
    new_df["driver_cancellation_reason"] = new_df["driver_cancellation_reason"].astype("category")
    new_df["cancelled_rides_by_driver"] = pd.to_numeric(new_df["cancelled_rides_by_driver"], errors="coerce")
    new_df["cancelled_rides_by_customer"] = pd.to_numeric(new_df["cancelled_rides_by_customer"], errors="coerce")
    new_df["incomplete_rides"] = pd.to_numeric(new_df["incomplete_rides"], errors="coerce")
    new_df["incomplete_rides_reason"] = new_df["incomplete_rides_reason"].astype("category")
    new_df["booking_value"] = pd.to_numeric(new_df["booking_value"], errors="coerce")
    new_df["ride_distance"] = pd.to_numeric(new_df["ride_distance"], errors="coerce")
    new_df["driver_ratings"] = pd.to_numeric(new_df["driver_ratings"], errors="coerce")
    new_df["customer_rating"] = pd.to_numeric(new_df["customer_rating"], errors="coerce")
    new_df["payment_method"] = new_df["payment_method"].astype("category")

    return new_df


df = rename_columns(df_raw)
df = cast_to_dtypes(df)

In [ ]:
def extract_temporal_features(df):
    new_df = df.copy()
    new_df["hour"] = new_df["datetime"].dt.hour
    new_df["day"] = new_df["datetime"].dt.day
    new_df["month"] = new_df["datetime"].dt.month
    new_df["weekday"] = new_df["datetime"].dt.dayofweek  # Monday=0, Sunday=6
    new_df["is_weekend"] = new_df["weekday"].isin([5, 6])

    def segment_time_of_day(hour):
        if 6 <= hour <= 11:
            return "morning"
        elif 12 <= hour <= 16:
            return "afternoon"
        elif 17 <= hour <= 22:
            return "evening"
        else:
            return "night"

    new_df["time_of_day"] = new_df["hour"].apply(lambda h: segment_time_of_day(h) if pd.notnull(h) else None)

    return new_df


df = rename_columns(df_raw)
df = cast_to_dtypes(df)
df = extract_temporal_features(df)

# Check result
df[["datetime", "hour", "day", "month", "weekday", "is_weekend", "time_of_day"]].head()

In [ ]:
df = rename_columns(df_raw)
df = cast_to_dtypes(df)
df = extract_temporal_features(df)

# Check data types and missing values
print("📌 Data Info:")
df.info()

print("\n📊 Summary Statistics:")
print(df.describe(include="all"))

print("\n🔍 Null Values per Column:")
print(df.isnull().sum())

In [ ]:
df = rename_columns(df_raw)
df = cast_to_dtypes(df)
df = extract_temporal_features(df)


# Create flags from cancellation and incomplete ride columns, null meaning it was NOT cancelled or incomplete
def cancelled_to_flag(df):
    new_df = df.copy()

    columns_to_flag = [
        "cancelled_rides_by_driver",
        "cancelled_rides_by_customer",
        "incomplete_rides",
    ]

    for col in columns_to_flag:
        flag_col = f"{col}_flag"
        new_df[flag_col] = new_df[col].notnull()

    return new_df


def missing_to_flag(df):
    new_df = df.copy()

    columns_to_flag = [
        "driver_ratings",
        "customer_rating",
        "booking_value",
        "payment_method",
    ]

    for col in columns_to_flag:
        flag_col = f"{col}_missing_flag"
        new_df[flag_col] = new_df[col].isnull()

    return new_df


df = cancelled_to_flag(df)
df = missing_to_flag(df)

print(
    df[
        [
            "cancelled_rides_by_driver_flag",
            "cancelled_rides_by_customer_flag",
            "incomplete_rides_flag",
            "driver_ratings_missing_flag",
            "customer_rating_missing_flag",
            "booking_value_missing_flag",
            "payment_method_missing_flag",
        ]
    ].sum()
)

In [ ]:
print(df.columns)

# EDA

## Temporal Features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# EDA
df = rename_columns(df_raw)
df = cast_to_dtypes(df)
df = extract_temporal_features(df)
df = cancelled_to_flag(df)
df = missing_to_flag(df)

df["customer_cancelled_target"] = df["cancelled_rides_by_customer_flag"].astype(int)

sns.set_theme(style="darkgrid", palette="pastel")

# 1. Cancellation Rate by Vehicle Type
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x="vehicle_type", y="customer_cancelled_target", hue="vehicle_type")
plt.title("Customer Cancellation Rate by Vehicle Type")
plt.xticks(rotation=45)
plt.show()

# 2. Cancellation Rate by Hour
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x="hour", y="customer_cancelled_target", hue="hour")
plt.title("Customer Cancellation Rate by Hour of Day")
plt.show()

# 3. Cancellation Rate by Day of Week
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x="weekday", y="customer_cancelled_target", hue="weekday")
plt.title("Customer Cancellation Rate by Day of Week")
plt.xticks(ticks=range(7), labels=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])
plt.show()

# 4. Cancellation Rate by Time of Day
plt.figure(figsize=(10, 5))
sns.barplot(
    data=df,
    x="time_of_day",
    y="customer_cancelled_target",
    order=["morning", "afternoon", "evening", "night"],
    hue="time_of_day",
)
plt.title("Customer Cancellation Rate by Time of Day")
plt.show()


# 5. Trend of Cancellations Over Time (Daily)
df_daily = df.groupby("date")["customer_cancelled_target"].mean().reset_index()

plt.figure(figsize=(14, 6))
sns.lineplot(data=df_daily, x="date", y="customer_cancelled_target")
plt.title("Daily Customer Cancellation Rate Over Time")
plt.ylabel("Cancellation Rate")
plt.xlabel("Date")
plt.show()

# 6. Optional: Heatmap of Hour vs Day of Week
heatmap_data = df.pivot_table(index="hour", columns="weekday", values="customer_cancelled_target", aggfunc="mean")

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap="YlGnBu", annot=True, fmt=".2f")
plt.title("Cancellation Rate by Hour and Day of Week")
plt.ylabel("Hour of Day")
plt.xlabel("Day of Week (0=Mon, 6=Sun)")
plt.show()

## Location Features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# EDA
df = rename_columns(df_raw)
df = cast_to_dtypes(df)
df = extract_temporal_features(df)
df = cancelled_to_flag(df)
df = missing_to_flag(df)

df["customer_cancelled_target"] = df["cancelled_rides_by_customer_flag"].astype(int)

sns.set_theme(style="darkgrid", palette="pastel")

# 1. Cancellation rate by top 10 pickup locations ordered from high to low
cancellation_rates = df.groupby("pickup_location", observed=True)["customer_cancelled_target"].mean()
top_50_locations = cancellation_rates.sort_values(ascending=False).head(50)
cumulative_sum = top_50_locations.cumsum()

df_plot = top_50_locations.reset_index(name="cancellation_rate")
df_plot["cumulative_cancellation_rate"] = cumulative_sum.values
# Create a figure and a primary axis
fig, ax1 = plt.subplots(figsize=(20, 10))

# Plot the bar chart on the primary axis
sns.barplot(
    x="pickup_location",
    y="cancellation_rate",
    data=df_plot,
    ax=ax1,
)

# Set the primary axis labels and title
ax1.set_xlabel("Pickup Location", fontsize=12)
ax1.set_ylabel("Cancellation Rate", fontsize=12)
ax1.set_title("Top 50 Pickup Locations by Cancellation Rate and Cumulative Sum", fontsize=16)
ax1.tick_params(axis="x", rotation=90)

# Create a secondary axis for the line plot
ax2 = ax1.twinx()

# Plot the cumulative sum line on the secondary axis
sns.lineplot(
    x="pickup_location",
    y="cumulative_cancellation_rate",
    data=df_plot,
    ax=ax2,
    color="red",
    marker="o",
    label="Cumulative Sum",
)

# Set the secondary axis label
ax2.set_ylabel("Cumulative Sum of Cancellation Rate", fontsize=12)
ax2.legend(loc="upper right")
plt.tight_layout()
plt.show()

# Feature Engineering

In [ ]:
# EDA
df = rename_columns(df_raw)
df = cast_to_dtypes(df)
df = extract_temporal_features(df)
df = cancelled_to_flag(df)
feature_df = missing_to_flag(df)


def calculate_rolling_counts(df, window, column_name):
    new_df = df = df.sort_values(by=["customer_id", "datetime"]).reset_index(drop=True)

    # Perform the rolling calculation on a temporary DataFrame with a datetime index
    temp_df = new_df.set_index("datetime")
    rolling_counts = temp_df.groupby("customer_id")["booking_id"].rolling(window=window).count()

    # The result has a MultiIndex (customer_id, datetime). Reset it to columns.
    rolling_counts = rolling_counts.reset_index(name=column_name)

    # We now have a temporary DataFrame with customer_id, datetime, and the count.
    # We need to shift the count to represent "previous" bookings.
    rolling_counts[column_name] = rolling_counts.groupby("customer_id")[column_name].shift(1).fillna(0).astype(int)

    return rolling_counts


feature_df["customer_cancelled_target"] = feature_df["cancelled_rides_by_customer_flag"].astype(int)

# 1. Vehicle Type Encoding
feature_df = pd.get_dummies(feature_df, columns=["vehicle_type"], prefix="vehicle", drop_first=True)

# 2. weekday encoding
feature_df = pd.get_dummies(feature_df, columns=["weekday"], prefix="weekday", drop_first=True)

# 3. time_of_day encoding
feature_df = pd.get_dummies(feature_df, columns=["time_of_day"], prefix="tod", drop_first=True)

# 4. Bookings prev week, prev_month, prev_year
# Calculate the rolling counts for each window
bookings_prev_week = calculate_rolling_counts(feature_df, "7D", "bookings_prev_week")
bookings_prev_month = calculate_rolling_counts(feature_df, "30D", "bookings_prev_month")
bookings_prev_year = calculate_rolling_counts(feature_df, "365D", "bookings_prev_year")

# Merge the calculated columns back into the original DataFrame
feature_df = pd.merge(feature_df, bookings_prev_week, on=["customer_id", "datetime"], how="left")
feature_df = pd.merge(feature_df, bookings_prev_month, on=["customer_id", "datetime"], how="left")
feature_df = pd.merge(feature_df, bookings_prev_year, on=["customer_id", "datetime"], how="left")

dropped_cols = [
    "booking_id",
    "customer_id",
    "pickup_location",
    "drop_location",
    "cancelled_rides_by_customer",
    "reason_for_cancelling_by_customer",
    "cancelled_rides_by_driver",
    "driver_cancellation_reason",
    "incomplete_rides",
    "incomplete_rides_reason",
    "datetime",
    "hour",
    "day",
    "month",
    "cancelled_rides_by_driver_flag",
    "cancelled_rides_by_customer_flag",
    "incomplete_rides_flag",
    "driver_ratings_missing_flag",
    "customer_rating_missing_flag",
    "booking_value_missing_flag",
    "payment_method_missing_flag",
]

feature_df = rename_columns(feature_df)
feature_df = feature_df.drop(columns=dropped_cols)
target = "customer_cancelled_target"
features = [col for col in feature_df if col != target]

print(f"Total Features for Modelling: {len(features)}")
print(features)

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights using sklearn
class_weights = compute_class_weight(class_weight="balanced", classes=np.array([0, 1]), y=feature_df[target])

weights = dict(enumerate(class_weights))
print("Class Weights:", weights)

In [ ]:
import numpy as np

# Combine X and y temporarily
temp_df = feature_df.copy()

boolean_cols = temp_df.select_dtypes(include="bool").columns

for col in boolean_cols:
    temp_df[col] = temp_df[col].astype(int)

numeric_cols = temp_df.select_dtypes(include=["number", "bool"]).columns

# Compute correlation with target
correlations = temp_df[numeric_cols].corr()[target].sort_values(key=abs, ascending=False)

# Display top correlations (excluding target itself)
print("📌 Top Correlated Features with Target:\n")
print(correlations)

# Modelling

## Baseline - Logistic Regression

In [ ]:
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

feature_df_copy = feature_df.copy()

boolean_cols = feature_df_copy.select_dtypes(include="bool").columns

for col in boolean_cols:
    feature_df_copy[col] = feature_df_copy[col].astype(int)

# Drop target and keep only numeric features
clean_features = feature_df_copy.select_dtypes(include=["number"]).columns.drop(target)

# Setup X and y
X = feature_df_copy[clean_features]
y = feature_df_copy[target]

# Impute missing values only for numeric columns
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

# Scaling
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print("🚂 Features we are training on!")
print(X.columns)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Train Logistic Regression
logreg = LogisticRegression(class_weight="balanced", max_iter=1000)
logreg.fit(X_train, y_train)

# Predict
y_pred = logreg.predict(X_test)
y_prob = logreg.predict_proba(X_test)[:, 1]

# Evaluate
print("🎯 ROC AUC Score:", roc_auc_score(y_test, y_prob))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred, digits=3))

## Model - RandomForest

In [ ]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

feature_df_copy = feature_df.copy()
feature_df_copy = feature_df_copy.drop(columns=["date", "time"])


features = feature_df_copy.columns
leaky_features = [
    "avg_vtat",
    "ride_distance",
    "driver_ratings",
    "customer_ratings",
    "booking_status",
    "payment_method",
    target,
]
clean_features = [f for f in features if f not in leaky_features]

# Prepare data
X = feature_df_copy[clean_features]
y = feature_df_copy[target]

# Impute missing values
imputer = SimpleImputer(strategy="median")

numeric_cols = X.select_dtypes(include=["number"]).columns
X_imputed = X.copy()
X_imputed[numeric_cols] = imputer.fit_transform(X[numeric_cols])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, stratify=y, random_state=42)

# Train Random Forest with balanced class weights
rf = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict & Evaluate
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:, 1]

print("🎯 ROC AUC Score:", roc_auc_score(y_test, y_prob))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred, digits=3))

## Model - XGBoost

In [ ]:
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

feature_df_copy = feature_df.copy()
feature_df_copy = feature_df_copy.drop(columns=["date", "time"])


features = feature_df_copy.columns
leaky_features = [
    "avg_vtat",
    "ride_distance",
    "driver_ratings",
    "customer_ratings",
    "booking_status",
    "payment_method",
    target,
]
clean_features = [f for f in features if f not in leaky_features]

# Prepare data
X = feature_df_copy[clean_features]
y = feature_df_copy[target]

# Impute missing values
imputer = SimpleImputer(strategy="median")

numeric_cols = X.select_dtypes(include=["number"]).columns
X_imputed = X.copy()
X_imputed[numeric_cols] = imputer.fit_transform(X[numeric_cols])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, stratify=y, random_state=42)

# Train XGBoost model
xgb_model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=13.29,
)

xgb_model.fit(X_train, y_train)

# Predict & Evaluate
y_pred = xgb_model.predict(X_test)
y_prob = xgb_model.predict_proba(X_test)[:, 1]

print("🎯 ROC AUC Score:", roc_auc_score(y_test, y_prob))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred, digits=3))